<a href="https://colab.research.google.com/github/MattiaVerticchio/PersonalProjects/blob/master/MarketForecasting/DriverlessAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Driverless AI instance

In [ ]:
# Driverless AI license key
LICENSE_KEY = 'Your license key'

# ngrok tunnel key
NGROK_AUTH = 'Your ngrok key'

In [ ]:
%%capture

# Driverless AI version and install script paths
DAI_SH = 'dai-1.9.0.3-linux-x86_64.sh'
PATH = 'https://s3.amazonaws.com/artifacts.h2o.ai/releases/ai/'\
       'h2o/dai/rel-1.9.0-16/x86_64-centos7/'
DAI_URL = PATH + DAI_SH

# Install folder and license file
! mkdir /root/.driverlessai
! echo $LICENSE_KEY >> /root/.driverlessai/license.sig

# Update the repositories
! apt update
# Install Driverless AI dependencies
! apt install -y  axel graphviz libcairo2 libpango1.0-0 opencl-headers clinfo ocl-icd-opencl-dev

# Set up Nvidia OpenCL
! mkdir -p /etc/OpenCL/vendors
! echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd

# Download and run the Driverless AI script
! axel -q $DAI_URL
! chmod 755 $DAI_SH && ./$DAI_SH

In [ ]:
%%capture

# Install ngrok
NGROK_PATH = 'https://bin.equinox.io/c/4VmDzA7iaHb/'
NGROK_ZIP = 'ngrok-stable-linux-amd64.zip'
NGROK_URL = NGROK_PATH + NGROK_ZIP
! axel -q $NGROK_URL
! unzip -qq -n $NGROK_ZIP

In [ ]:
from time import sleep
from IPython.display import clear_output
import requests
import json

# keep everything running
i=0
while True:
  try:
    # start driverless
    if len(get_ipython().getoutput('ps -ef | grep h2oai')) <= 2:
      ! nvidia-smi -pm 1
      ! /content/dai-1.9.0.3-linux-x86_64/run-dai.sh
      sleep(5)
      
    ngrok_link = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    print(ngrok_link)
    print("Username: h2oai Password: h2oai")  # Defaults
    sleep(10)
    requests.get(ngrok_link)
  except (KeyboardInterrupt, SystemExit):
      raise
  except:
    # create ngrok port
    get_ipython().system_raw('./ngrok authtoken $NGROK_AUTH && ./ngrok http 12345 &')
  i+=1
  if i >=5:
    clear_output(wait=True)
    i=0
